In [40]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
import pickle
import sklearn
import piton.datasets
from piton.featurizers import save_to_file, load_from_file
import sklearn.linear_model
import sklearn.metrics
import numpy as np
import os
import xgboost as xgb
import lightgbm as lgbm
import pandas as pd

In [54]:
features_path = "/share/pi/nigam/rthapa84/data/hba1c_reps"
labels_path = "/share/pi/nigam/rthapa84/data/HighHbA1c_labeled_patients_v3.pickle"
data_path = "/share/pi/nigam/data/som-rit-phi-starr-prod.starr_omop_cdm5_deid_2022_09_05_extract2"

data = piton.datasets.PatientDatabase(data_path)

#### CLMBR Model Representation and Performance

In [4]:
with open(features_path, "rb") as f:
    features = pickle.load(f)

with open(labels_path, "rb") as f:
    labeled_patients = pickle.load(f)

data_matrix, patient_ids, labeling_time = [
    features[k] for k in ("data_matrix", "patient_ids", "labeling_time")
]
labels = []

for pid, time in zip(patient_ids, labeling_time):
    raw_labels = [
        label.value for label in labeled_patients[pid] if label.time == time
    ]
    assert len(raw_labels) == 1
    labels.append(raw_labels[0])

labels = np.array(labels)

In [5]:
hashes = np.array([data.compute_split(97, pid) for pid in patient_ids])
train_pids = hashes < 70
val_pids = (hashes >= 70) & (hashes < 85)

In [6]:
if True:
    model = sklearn.linear_model.LogisticRegressionCV()
    model.fit(data_matrix[train_pids, :], labels[train_pids])

    with open("temp_model.pickle", "wb") as f:
        pickle.dump(model, f)
else:
    with open("temp_model.pickle", "rb") as f:
        model = pickle.load(f)

/share/pi/nigam/rthapa84/miniconda3/envs/env_piton_1/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/share/pi/nigam/rthapa84/miniconda3/envs/env_piton_1/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://s

In [7]:
train_preds = model.predict_log_proba(data_matrix[train_pids, :])[:, 1]
print(sklearn.metrics.roc_auc_score(labels[train_pids], train_preds))
print(sklearn.metrics.average_precision_score(labels[train_pids], train_preds))

0.9359589499336779
0.7517387202337262


In [8]:
val_preds = model.predict_log_proba(data_matrix[val_pids, :])[:, 1]
print(sklearn.metrics.roc_auc_score(labels[val_pids], val_preds))
print(sklearn.metrics.average_precision_score(labels[val_pids], val_preds))

0.9266718509509135
0.7272804418642


#### Performance with only Diabetes Text Embeddings

In [24]:
path_to_diabetes_shards = "/share/pi/nigam/rthapa84/data/diabetes_shards"
path_to_diabetes_metadata = "/share/pi/nigam/rthapa84/data/v1_diabetes_meta_data.pickle"

os.listdir(path_to_diabetes_shards)

['9_embeddings.pickle',
 '2_embeddings.pickle',
 '5_embeddings.pickle',
 '3_embeddings.pickle',
 '6_embeddings.pickle',
 '4_embeddings.pickle',
 '8_embeddings.pickle',
 '7_embeddings.pickle',
 '1_embeddings.pickle',
 '0_embeddings.pickle']

In [20]:
test_embedding = load_from_file(os.path.join(path_to_diabetes_shards, "0_embeddings.pickle"))

In [21]:
array_list = []

for file_name in os.listdir(path_to_diabetes_shards):
    test_embedding = load_from_file(os.path.join(path_to_diabetes_shards, file_name))
    print(test_embedding.shape)
    array_list.append(test_embedding)

(41846, 768)
(41847, 768)
(41846, 768)
(41846, 768)
(41846, 768)
(41846, 768)
(41846, 768)
(41846, 768)
(41848, 768)
(41848, 768)


In [22]:
embeddings = np.concatenate(array_list)
embeddings.shape

(418465, 768)

In [25]:
diabetes_meta_data = load_from_file(path_to_diabetes_metadata)

In [26]:
diabetes_meta_data

(array([False, False, False, ..., False, False, False]),
 array([     10,      16,      29, ..., 2730362, 2730370, 2730378]),
 array([datetime.datetime(2019, 10, 10, 7, 49),
        datetime.datetime(2021, 7, 3, 10, 23),
        datetime.datetime(2017, 10, 13, 11, 11), ...,
        datetime.datetime(2022, 9, 29, 23, 58),
        datetime.datetime(2022, 9, 11, 8, 17),
        datetime.datetime(2022, 8, 2, 16, 5)], dtype=object))

In [23]:
len(labeled_patients)

418465

In [29]:
diabetes_text_features = {
    "data_matrix": embeddings, 
    "labels": diabetes_meta_data[0],
    "patient_ids": diabetes_meta_data[1], 
    "labeling_time": diabetes_meta_data[2], 
}

#### Performance with just text data

In [31]:
data_matrix, labels, patient_ids, labeling_time = [
    diabetes_text_features[k] for k in ("data_matrix", "labels", "patient_ids", "labeling_time")
]

In [32]:
# Logistic Regression
model = sklearn.linear_model.LogisticRegressionCV()
model.fit(data_matrix[train_pids, :], labels[train_pids])
train_preds = model.predict_log_proba(data_matrix[train_pids, :])[:, 1]
print(sklearn.metrics.roc_auc_score(labels[train_pids], train_preds))
print(sklearn.metrics.average_precision_score(labels[train_pids], train_preds))

val_preds = model.predict_log_proba(data_matrix[val_pids, :])[:, 1]
print(sklearn.metrics.roc_auc_score(labels[val_pids], val_preds))
print(sklearn.metrics.average_precision_score(labels[val_pids], val_preds))

/share/pi/nigam/rthapa84/miniconda3/envs/env_piton_1/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/share/pi/nigam/rthapa84/miniconda3/envs/env_piton_1/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://s

LogisticRegressionCV()

In [36]:
train_preds = model.predict_log_proba(data_matrix[train_pids, :])[:, 1]
print(sklearn.metrics.roc_auc_score(labels[train_pids], train_preds))
print(sklearn.metrics.average_precision_score(labels[train_pids], train_preds))

val_preds = model.predict_log_proba(data_matrix[val_pids, :])[:, 1]
print(sklearn.metrics.roc_auc_score(labels[val_pids], val_preds))
print(sklearn.metrics.average_precision_score(labels[val_pids], val_preds))

0.5204884862390979
0.16398090342708407
0.5010860047980064
0.15019260792534728


In [48]:
# XGB
model = xgb.XGBClassifier(early_stopping=True)
model.fit(data_matrix[train_pids, :], labels[train_pids])
# train_preds = model.predict_log_proba(data_matrix[train_pids, :])[:, 1]
# print(sklearn.metrics.roc_auc_score(labels[train_pids], train_preds))
# print(sklearn.metrics.average_precision_score(labels[train_pids], train_preds))

# val_preds = model.predict_log_proba(data_matrix[val_pids, :])[:, 1]
# print(sklearn.metrics.roc_auc_score(labels[val_pids], val_preds))
# print(sklearn.metrics.average_precision_score(labels[val_pids], val_preds))

[18:29:56] WARNING: ../src/learner.cc:767: 
Parameters: { "early_stopping" } are not used.



AttributeError: 'XGBClassifier' object has no attribute 'predict_log_proba'

In [49]:
train_preds = model.predict_proba(data_matrix[train_pids, :])[::,1]
print(sklearn.metrics.roc_auc_score(labels[train_pids], train_preds))
print(sklearn.metrics.average_precision_score(labels[train_pids], train_preds))

val_preds = model.predict_proba(data_matrix[val_pids, :])[::,1]
print(sklearn.metrics.roc_auc_score(labels[val_pids], val_preds))
print(sklearn.metrics.average_precision_score(labels[val_pids], val_preds))

0.7666604156363233
0.48267486924518466
0.5046467417195637
0.1525801592757522


In [52]:
# LGBM
model = lgbm.LGBMClassifier()
model.fit(data_matrix[train_pids, :], labels[train_pids])
train_preds = model.predict_proba(data_matrix[train_pids, :])[::,1]
print(sklearn.metrics.roc_auc_score(labels[train_pids], train_preds))
print(sklearn.metrics.average_precision_score(labels[train_pids], train_preds))

val_preds = model.predict_proba(data_matrix[val_pids, :])[::,1]
print(sklearn.metrics.roc_auc_score(labels[val_pids], val_preds))
print(sklearn.metrics.average_precision_score(labels[val_pids], val_preds))

0.6977781231881377
0.4033554251983399
0.5015582326464695
0.1507149500051403


#### Now, let us concatenate clmbr embeddigs with Diabetes text embeddings

In [55]:
features

{'data_path': '/local-scratch/nigam/projects/ethanid/som-rit-phi-starr-prod.starr_omop_cdm5_deid_2022_09_05_extract2',
 'model': 'hbai1c_clmbr',
 'data_matrix': array([[ 1.251  ,  0.829  , -0.3005 , ..., -0.2654 , -1.0625 , -1.57   ],
        [ 0.2666 , -0.46   ,  0.4758 , ..., -1.274  , -0.35   , -1.506  ],
        [ 0.2174 ,  0.916  ,  1.002  , ..., -0.7876 , -2.365  ,  2.283  ],
        ...,
        [-0.11145, -0.127  ,  0.3909 , ..., -0.1497 , -0.952  , -1.449  ],
        [ 0.4834 ,  0.4414 ,  0.8467 , ..., -1.583  , -1.766  , -1.697  ],
        [ 0.1954 , -0.4365 , -0.1444 , ..., -0.385  ,  0.3882 ,  0.08673]],
       dtype=float16),
 'patient_ids': array([1534234, 1462978, 2403462, ..., 1733051, 2002114, 1688664],
       dtype=uint32),
 'labeling_time': array([datetime.datetime(2017, 7, 2, 10, 56),
        datetime.datetime(2018, 9, 28, 9, 48),
        datetime.datetime(2020, 12, 12, 13, 40), ...,
        datetime.datetime(2011, 8, 6, 9, 14),
        datetime.datetime(2022, 4, 24

In [61]:
features["data_matrix"].shape

(418465, 768)

In [65]:
clmbr_df = pd.DataFrame(features["data_matrix"])
clmbr_df["patient_ids"] = features["patient_ids"]
clmbr_df["labeling_time"] = features["labeling_time"]

In [66]:
text_df = pd.DataFrame(diabetes_text_features["data_matrix"])
text_df["patient_ids"] = diabetes_text_features["patient_ids"]
text_df["labeling_time"] = diabetes_text_features["labeling_time"]

In [67]:
text_df.head()

,0,1,2,3,4,5,6,7,8,9,...,760,761,762,763,764,765,766,767,patient_ids,labeling_time
0,-0.098187,0.135420,-0.048281,-0.072275,0.344851,-0.450779,0.169216,0.055684,0.235977,-0.035512,...,-0.401641,0.140419,-0.095032,0.020823,0.538744,-0.041634,0.138535,0.230650,10,2019-10-10 07:49:00
1,-0.280949,0.108300,-0.246979,-0.127010,0.195445,-0.255899,0.232926,0.257219,0.054682,-0.073611,...,-0.067651,-0.122727,0.008352,-0.013334,0.267678,-0.033346,-0.122832,-0.034070,16,2021-07-03 10:23:00
2,0.301556,-0.007186,0.012263,0.416464,0.176590,-0.286634,-0.290396,0.181623,0.011599,-0.211953,...,-0.331350,0.111225,-0.105274,0.122463,0.183359,-0.390962,-0.162274,-0.094727,29,2017-10-13 11:11:00
3,-0.060291,-0.020745,-0.003562,0.019252,0.870252,-0.073079,-0.014515,0.095009,0.137114,0.138180,...,-0.268811,-0.065248,-0.098902,0.116166,0.442825,0.024319,-0.034883,0.169472,65,2017-04-18 11:35:00
4,-0.069657,0.024101,0.003369,0.235734,0.589897,-0.232555,-0.099616,0.091591,0.014798,0.028854,...,-0.053344,-0.051040,-0.207480,0.002614,0.199728,-0.063617,-0.177353,0.275775,134,2022-01-14 07:43:00


In [68]:
clmbr_df.head()

,0,1,2,3,4,5,6,7,8,9,...,760,761,762,763,764,765,766,767,patient_ids,labeling_time
0,1.250977,0.829102,-0.300537,0.155640,0.175537,-0.839844,0.873047,1.913086,0.476318,-0.191772,...,-0.148682,0.003235,-0.778809,-0.149536,0.249634,-0.265381,-1.062500,-1.570312,1534234,2017-07-02 10:56:00
1,0.266602,-0.459961,0.475830,-0.126587,1.222656,-0.919922,0.890625,0.039673,0.045685,0.147949,...,-0.179932,0.692871,0.360840,1.055664,0.292480,-1.274414,-0.350098,-1.505859,1462978,2018-09-28 09:48:00
2,0.217407,0.916016,1.001953,1.145508,0.429199,0.555664,0.055664,1.373047,-1.906250,-0.986328,...,-1.753906,-0.973145,-0.432617,-0.021729,-1.649414,-0.787598,-2.365234,2.283203,2403462,2020-12-12 13:40:00
3,0.140015,-0.418701,0.151245,0.817871,1.521484,-1.934570,1.008789,0.068359,-0.662109,-1.147461,...,-0.135742,0.421143,-0.308105,0.035583,-1.610352,0.424805,0.800293,-1.634766,1863650,2021-07-20 14:57:00
4,0.399902,0.732910,-0.119263,0.442871,0.771484,-1.844727,0.902344,1.985352,0.320801,-0.223267,...,-1.110352,-1.389648,0.857422,-0.589355,-2.078125,-1.000977,-0.614746,-0.558105,2426984,2021-06-18 10:20:00


In [72]:
clmbr_df.shape, text_df.shape

((418465, 770), (418465, 770))

In [69]:
combined_df = pd.merge(clmbr_df, text_df, on=['patient_ids','labeling_time'])

In [70]:
combined_df.shape

(418465, 1538)

In [71]:
combined_df.head()

,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,...,758_y,759_y,760_y,761_y,762_y,763_y,764_y,765_y,766_y,767_y
0,1.250977,0.829102,-0.300537,0.155640,0.175537,-0.839844,0.873047,1.913086,0.476318,-0.191772,...,0.088929,0.050944,-0.287264,0.052080,-0.100184,0.064651,0.358182,-0.323161,-0.048184,0.051253
1,0.266602,-0.459961,0.475830,-0.126587,1.222656,-0.919922,0.890625,0.039673,0.045685,0.147949,...,-0.016201,-0.006076,-0.154375,0.078132,-0.016915,0.023570,0.193341,-0.267965,-0.103342,0.187395
2,0.217407,0.916016,1.001953,1.145508,0.429199,0.555664,0.055664,1.373047,-1.906250,-0.986328,...,0.071850,-0.086112,-0.089172,0.054974,-0.129177,0.083780,0.159389,-0.333712,-0.045663,0.138630
3,0.140015,-0.418701,0.151245,0.817871,1.521484,-1.934570,1.008789,0.068359,-0.662109,-1.147461,...,0.083221,0.145158,-0.287627,-0.023387,0.030881,0.092805,0.461680,-0.265873,-0.185706,0.093875
4,0.399902,0.732910,-0.119263,0.442871,0.771484,-1.844727,0.902344,1.985352,0.320801,-0.223267,...,0.022662,-0.069294,-0.305145,-0.215065,-0.201919,0.239134,-0.226174,-0.307096,-0.183732,0.268839


In [91]:
combined_data_matirx = combined_df.drop(columns=['patient_ids','labeling_time']).to_numpy()
patient_ids = combined_df["patient_ids"].to_numpy()
labeling_time = pd.to_datetime(combined_df["labeling_time"]).to_numpy()

In [92]:
labeling_time

array(['2017-07-02T10:56:00.000000000', '2018-09-28T09:48:00.000000000',
       '2020-12-12T13:40:00.000000000', ...,
       '2011-08-06T09:14:00.000000000', '2022-04-24T10:13:00.000000000',
       '2019-04-23T06:15:00.000000000'], dtype='datetime64[ns]')

In [79]:
combined_features = {
    "data_matrix": combined_data_matirx, 
    "patient_ids": patient_ids, 
    "labeling_time": labeling_time
}

In [93]:
with open(labels_path, "rb") as f:
    labeled_patients = pickle.load(f)
    
data_matrix, patient_ids, labeling_time = [
    combined_features[k] for k in ("data_matrix", "patient_ids", "labeling_time")
]
labels = []

for pid, time in zip(patient_ids, labeling_time):
    raw_labels = [
        label.value for label in labeled_patients[pid]
    ]
    assert len(raw_labels) == 1
    labels.append(raw_labels[0])

labels = np.array(labels)

In [94]:
len(labels)

418465

In [95]:
hashes = np.array([data.compute_split(97, pid) for pid in patient_ids])
train_pids = hashes < 70
val_pids = (hashes >= 70) & (hashes < 85)

In [96]:
# Logistic Regression
model = sklearn.linear_model.LogisticRegressionCV()
model.fit(data_matrix[train_pids, :], labels[train_pids])

/share/pi/nigam/rthapa84/miniconda3/envs/env_piton_1/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/share/pi/nigam/rthapa84/miniconda3/envs/env_piton_1/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://s

LogisticRegressionCV()

In [97]:
train_preds = model.predict_log_proba(data_matrix[train_pids, :])[:, 1]
print(sklearn.metrics.roc_auc_score(labels[train_pids], train_preds))
print(sklearn.metrics.average_precision_score(labels[train_pids], train_preds))

val_preds = model.predict_log_proba(data_matrix[val_pids, :])[:, 1]
print(sklearn.metrics.roc_auc_score(labels[val_pids], val_preds))
print(sklearn.metrics.average_precision_score(labels[val_pids], val_preds))

0.9362383067354654
0.7529105576620478
0.9265394108918775
0.7272681479319956


In [98]:
# XGB
model = xgb.XGBClassifier(early_stopping=True)
model.fit(data_matrix[train_pids, :], labels[train_pids])

train_preds = model.predict_proba(data_matrix[train_pids, :])[::,1]
print(sklearn.metrics.roc_auc_score(labels[train_pids], train_preds))
print(sklearn.metrics.average_precision_score(labels[train_pids], train_preds))

val_preds = model.predict_proba(data_matrix[val_pids, :])[::,1]
print(sklearn.metrics.roc_auc_score(labels[val_pids], val_preds))
print(sklearn.metrics.average_precision_score(labels[val_pids], val_preds))

[19:23:47] WARNING: ../src/learner.cc:767: 
Parameters: { "early_stopping" } are not used.

0.9696063285396885
0.8672684992517792
0.9225599300260252
0.7115358477865467


In [99]:
# LGBM
model = lgbm.LGBMClassifier()
model.fit(data_matrix[train_pids, :], labels[train_pids])
train_preds = model.predict_proba(data_matrix[train_pids, :])[::,1]
print(sklearn.metrics.roc_auc_score(labels[train_pids], train_preds))
print(sklearn.metrics.average_precision_score(labels[train_pids], train_preds))

val_preds = model.predict_proba(data_matrix[val_pids, :])[::,1]
print(sklearn.metrics.roc_auc_score(labels[val_pids], val_preds))
print(sklearn.metrics.average_precision_score(labels[val_pids], val_preds))

0.9468478412352219
0.7751813294412079
0.9246340531673541
0.7224326926429762


#### Performance with only Mortality Text Embeddings

In [121]:
path_to_mortality_shards = "/share/pi/nigam/rthapa84/data/mortality_shards"
path_to_mortality_metadata = "/share/pi/nigam/rthapa84/data/v1_mortality_meta_data.pickle"
labels_path = "/share/pi/nigam/rthapa84/data/mortality_labeled_patients_v1.pickle"

os.listdir(path_to_diabetes_shards)

['9_embeddings.pickle',
 '2_embeddings.pickle',
 '5_embeddings.pickle',
 '3_embeddings.pickle',
 '6_embeddings.pickle',
 '4_embeddings.pickle',
 '8_embeddings.pickle',
 '7_embeddings.pickle',
 '1_embeddings.pickle',
 '0_embeddings.pickle']

In [122]:
array_list = []

for file_name in os.listdir(path_to_mortality_shards):
    test_embedding = load_from_file(os.path.join(path_to_mortality_shards, file_name))
    print(test_embedding.shape)
    array_list.append(test_embedding)

(23764, 768)
(23766, 768)
(23764, 768)
(23766, 768)
(23764, 768)
(23766, 768)
(23764, 768)
(23764, 768)
(23766, 768)
(23766, 768)


In [123]:
embeddings = np.concatenate(array_list)
embeddings.shape

(237650, 768)

In [124]:
mortality_meta_data = load_from_file(path_to_mortality_metadata)

In [125]:
mortality_text_features = {
    "data_matrix": embeddings, 
    "labels": mortality_meta_data[0],
    "patient_ids": mortality_meta_data[1], 
    "labeling_time": mortality_meta_data[2], 
}

In [126]:
data_matrix, labels, patient_ids, labeling_time = [
    mortality_text_features[k] for k in ("data_matrix", "labels", "patient_ids", "labeling_time")
]

In [127]:
hashes = np.array([data.compute_split(97, pid) for pid in patient_ids])
train_pids = hashes < 70
val_pids = (hashes >= 70) & (hashes < 85)

In [128]:
val_pids.shape

(237650,)

In [129]:
# Logistic Regression
model = sklearn.linear_model.LogisticRegressionCV()
model.fit(data_matrix[train_pids, :], labels[train_pids])

/share/pi/nigam/rthapa84/miniconda3/envs/env_piton_1/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/share/pi/nigam/rthapa84/miniconda3/envs/env_piton_1/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://s

LogisticRegressionCV()

In [130]:
train_preds = model.predict_log_proba(data_matrix[train_pids, :])[:, 1]
print(sklearn.metrics.roc_auc_score(labels[train_pids], train_preds))
print(sklearn.metrics.average_precision_score(labels[train_pids], train_preds))

val_preds = model.predict_log_proba(data_matrix[val_pids, :])[:, 1]
print(sklearn.metrics.roc_auc_score(labels[val_pids], val_preds))
print(sklearn.metrics.average_precision_score(labels[val_pids], val_preds))

0.5397209503841645
0.05499512580133011
0.5073423567540761
0.05113908474695157


In [131]:
# XGB
model = xgb.XGBClassifier(early_stopping=True)
model.fit(data_matrix[train_pids, :], labels[train_pids])
train_preds = model.predict_proba(data_matrix[train_pids, :])[::,1]
print(sklearn.metrics.roc_auc_score(labels[train_pids], train_preds))
print(sklearn.metrics.average_precision_score(labels[train_pids], train_preds))

val_preds = model.predict_proba(data_matrix[val_pids, :])[::,1]
print(sklearn.metrics.roc_auc_score(labels[val_pids], val_preds))
print(sklearn.metrics.average_precision_score(labels[val_pids], val_preds))

[20:08:32] WARNING: ../src/learner.cc:767: 
Parameters: { "early_stopping" } are not used.

0.9500227304892654
0.7330945743486263
0.5052701661560737
0.05175069875845791


In [132]:
# LGBM
model = lgbm.LGBMClassifier()
model.fit(data_matrix[train_pids, :], labels[train_pids])
train_preds = model.predict_proba(data_matrix[train_pids, :])[::,1]
print(sklearn.metrics.roc_auc_score(labels[train_pids], train_preds))
print(sklearn.metrics.average_precision_score(labels[train_pids], train_preds))

val_preds = model.predict_proba(data_matrix[val_pids, :])[::,1]
print(sklearn.metrics.roc_auc_score(labels[val_pids], val_preds))
print(sklearn.metrics.average_precision_score(labels[val_pids], val_preds))

0.8717973313961981
0.5504784667762804
0.49728926582434063
0.05004677820577496


#### Let us combined CLMBR representation with Mortality text data

In [133]:
features_path = "/share/pi/nigam/rthapa84/data/mortality_clmbr_tuned"
labels_path = "/share/pi/nigam/rthapa84/data/mortality_labeled_patients_v1.pickle"

with open(features_path, "rb") as f:
    features = pickle.load(f)
    
with open(labels_path, "rb") as f:
    labeled_patients = pickle.load(f)

In [134]:
clmbr_df = pd.DataFrame(features["data_matrix"])
clmbr_df["patient_ids"] = features["patient_ids"]
clmbr_df["labeling_time"] = features["labeling_time"]

In [135]:
text_df = pd.DataFrame(mortality_text_features["data_matrix"])
text_df["patient_ids"] = mortality_text_features["patient_ids"]
text_df["labeling_time"] = mortality_text_features["labeling_time"]

In [136]:
combined_df = pd.merge(clmbr_df, text_df, on=['patient_ids','labeling_time'])

In [137]:
combined_data_matirx = combined_df.drop(columns=['patient_ids','labeling_time']).to_numpy()
patient_ids = combined_df["patient_ids"].to_numpy()
labeling_time = pd.to_datetime(combined_df["labeling_time"]).to_numpy()

In [138]:
combined_features = {
    "data_matrix": combined_data_matirx, 
    "patient_ids": patient_ids, 
    "labeling_time": labeling_time
}

In [139]:
with open(labels_path, "rb") as f:
    labeled_patients = pickle.load(f)
    
data_matrix, patient_ids, labeling_time = [
    combined_features[k] for k in ("data_matrix", "patient_ids", "labeling_time")
]
labels = []

for pid, time in zip(patient_ids, labeling_time):
    raw_labels = [
        label.value for label in labeled_patients[pid]
    ]
    assert len(raw_labels) == 1
    labels.append(raw_labels[0])

labels = np.array(labels)

In [140]:
hashes = np.array([data.compute_split(97, pid) for pid in patient_ids])
train_pids = hashes < 70
val_pids = (hashes >= 70) & (hashes < 85)

In [141]:
# Logistic Regression
model = sklearn.linear_model.LogisticRegressionCV()
model.fit(data_matrix[train_pids, :], labels[train_pids])

/share/pi/nigam/rthapa84/miniconda3/envs/env_piton_1/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/share/pi/nigam/rthapa84/miniconda3/envs/env_piton_1/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://s

LogisticRegressionCV()

In [142]:
train_preds = model.predict_log_proba(data_matrix[train_pids, :])[:, 1]
print(sklearn.metrics.roc_auc_score(labels[train_pids], train_preds))
print(sklearn.metrics.average_precision_score(labels[train_pids], train_preds))

val_preds = model.predict_log_proba(data_matrix[val_pids, :])[:, 1]
print(sklearn.metrics.roc_auc_score(labels[val_pids], val_preds))
print(sklearn.metrics.average_precision_score(labels[val_pids], val_preds))

0.9755669037398461
0.7819516977453588
0.9633667970780531
0.6986775598628536


In [143]:
# XGB
model = xgb.XGBClassifier()
model.fit(data_matrix[train_pids, :], labels[train_pids])

train_preds = model.predict_proba(data_matrix[train_pids, :])[::,1]
print(sklearn.metrics.roc_auc_score(labels[train_pids], train_preds))
print(sklearn.metrics.average_precision_score(labels[train_pids], train_preds))

val_preds = model.predict_proba(data_matrix[val_pids, :])[::,1]
print(sklearn.metrics.roc_auc_score(labels[val_pids], val_preds))
print(sklearn.metrics.average_precision_score(labels[val_pids], val_preds))

0.9998435059777518
0.9973982803664642
0.9584292265989477
0.6750637874085395


In [144]:
# LGBM
model = lgbm.LGBMClassifier()
model.fit(data_matrix[train_pids, :], labels[train_pids])
train_preds = model.predict_proba(data_matrix[train_pids, :])[::,1]
print(sklearn.metrics.roc_auc_score(labels[train_pids], train_preds))
print(sklearn.metrics.average_precision_score(labels[train_pids], train_preds))

val_preds = model.predict_proba(data_matrix[val_pids, :])[::,1]
print(sklearn.metrics.roc_auc_score(labels[val_pids], val_preds))
print(sklearn.metrics.average_precision_score(labels[val_pids], val_preds))

0.993542915482733
0.8941384576570273
0.9604157146717941
0.6712694986918075
